In [31]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils

In [32]:
import cv2
import matplotlib.pyplot as plt
from scipy.fftpack import dct, idct
from random import shuffle as shuf
from imutils import paths
import os
import numpy as np

In [33]:
# implement 2D DCT
def dct2(a):
    return dct(dct(a.T, norm='ortho').T, norm='ortho')

# implement 2D IDCT
def idct2(a):
    return idct(idct(a.T, norm='ortho').T, norm='ortho') 

def dct_compression(img):
    imf = np.float32(img)
    comp = dct2(imf)/255. # Scale to force low values to 0
    comp = np.uint8(idct2(comp)*255)
    return comp

In [34]:
def cv_disp_img(img, title="image"):
    while True:
        cv2.imshow(title, img)
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            # q key pressed so quit
            print("Quitting...")
            break

    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [35]:
imagePaths = list(paths.list_images("./Dataset/Train/"))

In [36]:
# Generate Dataset Matrix & Normalize/DCT
width = height = 32
size = (width * height)

image_mat = np.zeros((len(imagePaths), height,width),dtype=np.uint8)

for ct, image in enumerate(imagePaths):
    img = cv2.imread(image,0)
    img = cv2.resize(img, (height,width), interpolation=cv2.INTER_LINEAR)
    norm_img = np.zeros((height,width))
    norm_img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
    dct_image = dct_compression(img)
    image_mat[ct, :, :] = img

In [37]:
# 2DPCA
def give_P(eig_vals,quality):
    s = np.sum(eig_vals)
    thresh = s * quality/100
    t = 0
    P = 0
    while t < thresh:
        t += eig_vals[P]
        P += 1
        
    return P
        
def reduce_dim(images,mean_subtracted):
    num = images.shape[0]
    height = images.shape[1]
    width = images.shape[2]
    
    g_t = np.zeros((height,height))
    h_t = np.zeros((width,width))
    
    
    for i in range(num):
        temp_gt = np.dot(mean_subtracted[i].T,mean_subtracted[i])
        temp_ht = np.dot(mean_subtracted[i], mean_subtracted[i].T)
        
        g_t += temp_gt
        h_t += temp_ht
    
    g_t /= num
    h_t /= num
    
    #g_t
    g_e_vals, g_e_vec = np.linalg.eig(g_t)
    P_1 = give_P(eig_vals=g_e_vals,quality=90)
    print("P_1: ",P_1)
    new_bases_gt = g_e_vec[:,0:P_1]
    
    #h_t
    h_e_vals, h_e_vec = np.linalg.eig(h_t)
    P_2 = give_P(eig_vals=h_e_vals,quality=90)
    print("P_2: ",P_2)
    new_bases_ht = h_e_vec[:,0:P_2]
    
    new_cord_gt = np.dot(images, new_bases_gt)
    
    new_cord = np.zeros((num, P_2, P_1))
    
    for i in range(num):
        new_cord[i,:,:] = np.dot(new_bases_ht.T, new_cord_gt[i])
    
    
    return (new_cord, new_bases_gt, new_bases_ht)

Quitting...


In [38]:
mean_face = np.mean(image_mat,0)
images_mean_subtracted = image_mat - mean_face


new_cord,new_bases_gt, new_bases_ht = reduce_dim(image_mat,images_mean_subtracted)

P_1:  8
P_2:  5


In [58]:
np.save("./PP_Data/new_cord.npy", new_cord)
np.save("./PP_Data/new_bases_gt.npy", new_bases_gt)
np.save("./PP_Data/new_bases_ht.npy", new_bases_ht)

In [40]:
def image_to_feature_vector(image, size=(32, 32)):
	# resize the image to a fixed size, then flatten the image into
	# a list of raw pixel intensities
	return cv2.resize(image, size).flatten()

data = []
labels = []

for (i, imagePath) in enumerate(imagePaths):
	# load the image and extract the class label (assuming that our
	# path as the format: /path/to/dataset/{class}.{image_num}.jpg
	image = new_cord[i]
	label = imagePath.split(os.path.sep)[-1].split(".")[0]

	# construct a feature vector raw pixel intensities, then update
	# the data matrix and labels list
	features = image_to_feature_vector(image)
	data.append(features)
	labels.append(label)

	# show an update every 1,000 images
	if i > 0 and i % 10 == 0:
		print("[INFO] processed {}/{}".format(i, len(imagePaths)))

[INFO] processed 10/64
[INFO] processed 20/64
[INFO] processed 30/64
[INFO] processed 40/64
[INFO] processed 50/64
[INFO] processed 60/64


['Dad',
 'Mom',
 'Connor',
 'Connor',
 'Mom',
 'Dad',
 'Dad',
 'Mom',
 'Connor',
 'Connor',
 'Mom',
 'Dad',
 'Connor',
 'Dad',
 'Dad',
 'Mom',
 'Dad',
 'Dad',
 'Maddie',
 'Mom',
 'Maddie',
 'Maddie',
 'Mom',
 'Maddie',
 'Maddie',
 'Maddie',
 'Maddie',
 'Maddie',
 'Connor',
 'Maddie',
 'Maddie',
 'Connor',
 'Maddie',
 'Maddie',
 'Maddie',
 'Maddie',
 'Mom',
 'Connor',
 'Connor',
 'Mom',
 'Mom',
 'Connor',
 'Connor',
 'Mom',
 'Mom',
 'Maddie',
 'Connor',
 'Connor',
 'Mom',
 'Mom',
 'Connor',
 'Connor',
 'Mom',
 'Dad',
 'Dad',
 'Dad',
 'Dad',
 'Dad',
 'Dad',
 'Dad',
 'Maddie',
 'Maddie',
 'Maddie',
 'Maddie']

In [41]:
le = LabelEncoder()
labels = le.fit_transform(labels)

In [42]:
data = np.array(data) / 255.0
labels = np_utils.to_categorical(labels, 4)

In [59]:
np.save("./PP_Data/data.npy", data)
np.save("./PP_Data/labels.npy", labels)

In [44]:
(trainData, testData, trainLabels, testLabels) = train_test_split(
	data, labels, test_size=0.25, random_state=42)

In [45]:
model = Sequential()
model.add(Dense(768, input_dim=1024, init="uniform",
	activation="relu"))
model.add(Dense(384, activation="relu", kernel_initializer="uniform"))
model.add(Dense(4))
model.add(Activation("softmax"))

# train the model using SGD
print("[INFO] compiling model...")
sgd = SGD(lr=0.01)
model.compile(loss="binary_crossentropy", optimizer=sgd,
	metrics=["accuracy"])
model.fit(trainData, trainLabels, epochs=100, batch_size=128,
	verbose=1)

# show the accuracy on the testing set
print("[INFO] evaluating on testing set...")
(loss, accuracy) = model.evaluate(testData, testLabels,
	batch_size=128, verbose=1)
print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,
	accuracy * 100))

# dump the network architecture and weights to file
print("[INFO] dumping architecture and weights to file...")
model.save("./model.hdf5")

[INFO] compiling model...


/Users/connordepalma/anaconda3/envs/CPE646/lib/python3.7/site-packages/ipykernel_launcher.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(768, input_dim=1024, activation="relu", kernel_initializer="uniform")`
  This is separate from the ipykernel package so we can avoid doing imports until


Epoch 1/100
48/48 [==============================] - 1s 15ms/step - loss: 0.6751 - accuracy: 0.7083
Epoch 2/100
48/48 [==============================] - 0s 224us/step - loss: 0.4693 - accuracy: 0.7500
Epoch 3/100
48/48 [==============================] - 0s 221us/step - loss: 0.3795 - accuracy: 0.8125
Epoch 4/100
48/48 [==============================] - 0s 251us/step - loss: 0.3265 - accuracy: 0.8542
Epoch 5/100
48/48 [==============================] - 0s 226us/step - loss: 0.2893 - accuracy: 0.8698
Epoch 6/100
48/48 [==============================] - 0s 249us/step - loss: 0.2620 - accuracy: 0.9167
Epoch 7/100
48/48 [==============================] - 0s 219us/step - loss: 0.2416 - accuracy: 0.9219
Epoch 8/100
48/48 [==============================] - 0s 237us/step - loss: 0.2259 - accuracy: 0.9219
Epoch 9/100
48/48 [==============================] - 0s 230us/step - loss: 0.2132 - accuracy: 0.9271
Epoch 10/100
48/48 [==============================] - 0s 228us/step - loss: 0.2027 - accurac

In [46]:
from keras.models import load_model

In [47]:
CLASSES = ["Connor", "Maddie", "Dad", "Mom"]
model = load_model("./model.hdf5")

In [48]:
def recognize_faces():
    cv2.namedWindow("aha")
    
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")    
    
    webcam = cv2.VideoCapture(0)
    
    success = webcam.isOpened()
    if success == False:
        print('Error: Camera could not be opened')
    else:
        print('Success: Grabbed the camera')
    while True:
        ret, frame = webcam.read()
        label = ""
        if ret:
            faces = face_cascade.detectMultiScale(frame, scaleFactor=1.1, minNeighbors=5) 
            for (x,y,w,h) in faces:
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 1);

                p=10
                img = cv2.cvtColor(frame[y-p+1:y+h+p, x-p+1:x+w+p], cv2.COLOR_BGR2GRAY)
                img = cv2.resize(img, (32,32), interpolation=cv2.INTER_LINEAR)
                norm_img = np.zeros((32,32))
                norm_img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
                dct_image = dct_compression(img)
                face = np.dot(new_bases_ht.T,np.dot(dct_image,new_bases_gt))

                features = np.array([image_to_feature_vector(face)/255.])
                probs = model.predict(features)[0]
                prediction = probs.argmax(axis=0)



                label = "{}: {:.2f}%".format(CLASSES[prediction],
                probs[prediction] * 100)
                cv2.putText(frame, label, (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                            1.0, (0, 255, 0), 3)
            cv2.imshow("ahahaha", frame)
        
        if not ret:
            break
        # Monitor keystrokes
        k = cv2.waitKey(1)

        if k & 0xFF == ord('q'):
            print("Quitting...")
            break
       
    webcam.release()
    cv2.destroyAllWindows()
    cv2.waitKey(1)

In [49]:
recognize_faces()

Success: Grabbed the camera
Quitting...
